In [1]:
import os
os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'
from pixelmed_calc.medical_imaging.modelstore import get_model_from_name
from pixelmed_calc.medical_imaging.DLtools.process2d import load_data_from_csv,get_file_and_labels,get_transforms,CustomImageDataset,train_and_validate
import torch
from monai.data import DataLoader
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")  
model=get_model_from_name['resnet50_SE'](num_classes = 2, pretrained = False,in_channels=3).to("cuda")
#如果有预训练好的2D模型，则修改以下代码路径
model.load_state_dict(torch.load(r'd:\pixelmedAI\function_pm\note2-深度学习分类\results\latest_model.pth'))

train_base_path = r'D:\pixelmedAI\function_pm\pm_data\MR_jpg\images'
val_base_path = r'D:\pixelmedAI\function_pm\pm_data\MR_jpg\images'
input_size=224
# CSV文件路径
csv_path = r'D:\pixelmedAI\function_pm\pm_data\MR_jpg\group.csv'
tfs =get_transforms(input_size)

# 读取所有数据
file_names, labels, groups = load_data_from_csv(csv_path)

# 获取训练数据和标签（可以设置 base_path 为 None 或实际路径）
file_train, train_labels = get_file_and_labels(file_names, labels, groups, 'train', train_base_path)

# 获取验证数据和标签（可以设置 base_path 为 None 或实际路径）
file_val, val_labels = get_file_and_labels(file_names, labels, groups, 'test', val_base_path)

# 假设你已经定义了 ImageDataset 和 train_transforms, val_transforms
train_ds = CustomImageDataset(image_files=file_train, labels=train_labels, transform=tfs)
train_loader = DataLoader(train_ds, batch_size=4, shuffle=True, num_workers=0, pin_memory=torch.cuda.is_available())

val_ds = CustomImageDataset(image_files=file_val, labels=val_labels, transform=tfs)
val_loader = DataLoader(val_ds, batch_size=1, shuffle=False, num_workers=0, pin_memory=torch.cuda.is_available())



In [2]:
from pixelmed_calc.medical_imaging.DLtools.utils import save_prob_csv,eval_model
val_ds = CustomImageDataset(image_files=file_val, labels=val_labels,transform=tfs)  
val_loader = DataLoader(val_ds, batch_size=1, shuffle=False, num_workers=0, pin_memory=torch.cuda.is_available())

val_metrics = eval_model(model, val_loader, device)  
save_prob_csv(val_metrics, os.path.join('results', f'test1.csv'))  

The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
